<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/instrGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction Generation

## Basic includes

In [ ]:
!pip install ipython-autotime
%load_ext autotime
!pip install word2vec
!pip install ray[tune]

## Connect colab and set paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

rootDir = '/content/drive/MyDrive/'

TIMESTAMP = '2022_03_19'

dataPath = rootDir + 'TP2/Datasets/Recipe1M/' + TIMESTAMP
tarPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP


import sys
sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM')


In [1]:
# local exec
tarPath = 'D:/02_Studium/SBX/mad-recime/data/2022_03_19'

import os
sys.path.append(os.getcwd())

## Basic includes

In [2]:
import pandas as pd
import numpy as np

import torch

# Model
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
from torchsummary import summary

# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# hyperparameter tuning
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [3]:
import src.instGen as instGen
import src.preProc as preProc
import importlib

## Seed

In [4]:
torch.manual_seed(0)
np.random.seed(0)

## Training execution
mixture of 
* https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
* https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html
* https://stackoverflow.com/questions/67295494/correct-validation-loss-in-pytorch


### Set device

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print(torch.cuda.get_device_name(0))

### Set hyperparams

In [6]:
importlib.reload(instGen)

hyperParams = instGen.HyperParams(epochs=1, batchSize=2, hiddenDim=10, numLayers=1, embeddingDim=100)
print(hyperParams)

epochs 1
batchSize 2
lr 0.001
ratio train|val [0.7, 0.3]
hiddenDim 10
numLayers 1
embeddingDim 100



### Set dataset

In [7]:
importlib.reload(instGen)
importlib.reload(preProc)

instSet = instGen.InstructionSet(tarPath, setSize=2)

# split data set
trainNum = int(hyperParams.ratio[0] * len(instSet))
valNum = int(hyperParams.ratio[1] * len(instSet))


splitSet = random_split(
    instSet, [trainNum, valNum], generator=torch.Generator().manual_seed(0))
    
splitSet = {'train': splitSet[0], 'val': splitSet[1]}

testSet = instGen.getPreProcData(tarPath, range(-1,0))

d:\02_Studium\SBX\mad-recime\network\LSTM\src\preProc.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))
d:\02_Studium\SBX\mad-recime\network\LSTM\src\preProc.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))


In [8]:
pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_index))

,0
OOV,1
the,2
and,3
in,4
a,5
...,...
verde,1604
curd,1605
dj's,1606
dishes,1607


### Set model

In [9]:
importlib.reload(instGen)

model = instGen.EmbedLSTM(hyperParams, instSet, device)
model.to(device)
print(model)


EmbedLSTM(
  (word_embeddings): Embedding(1608, 100, padding_idx=0)
  (lstm): LSTM(100, 10, batch_first=True)
  (linear): Linear(in_features=10, out_features=1608, bias=True)
)


### Train

In [10]:
importlib.reload(instGen)

instGen.train(splitSet, model, hyperParams, device)

Epoch: 1, loss: 7.296695790972028, acc: 0.0185064935064935


### Test

In [17]:
importlib.reload(instGen)

idx = np.random.randint(0, len(testSet))
inputIngs = testSet['ingredient'].values[idx]
inputTitle = testSet['title'].values[idx].split()
seq = instGen.testInputPrep(inputIngs + inputTitle, instSet.tokenizer)
print(inputIngs)
print(inputTitle)
print(seq)
instGen.sample(model, instSet, 300, device, initial=seq)
# todo: get avg length of instructions

['boneless skinless chicken breasts', 'mustard seeds', 'cumin seeds', 'garam masala', 'olive oil', 'sweet potatoes', 'garlic cloves', 'mustard seeds', 'cumin seeds', 'garam masala', 'cayenne pepper', 'ginger', 'curry powder', 'red curry paste', 'tomato paste', 'cinnamon stick', 'heavy cream', 'salt and pepper']
['Chicken', 'and', 'Sweet', 'Potato', 'Curry']
[724, 725, 28, 261, 199, 101, 165, 101, 366, 367, 72, 29, 181, 79, 44, 147, 199, 101, 165, 101, 366, 367, 368, 24, 369, 331, 48, 138, 331, 370, 180, 370, 74, 424, 262, 35, 14, 3, 24, 28, 3, 181, 425, 331]
36
691
955
36
1493
955
84
955
960
200
226
1493
893
1054
45
1406
1493
955
84
955
960
1394
1406
1141
955
955
83
83
955
83
83
83
83
247
1121
1054
1257
1054
1141
955
955
1273
1273
955
1013
642
1055
932
297
667
503
1394
863
36
1533
231
133
960
833
1578
955
1013
642
1055
932
297
667
503
1394
863
36
1533
231
133
960
833
1578
955
1013
642
1055
932
297
667
503
1394
863
36
1533
231
133
960
833
1578
955
1013
642
1055
932
297
667
503
1394
863


'cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll fried olives chunk cheesie flatbread beaten drained biscuit so cabbage form thickened chilled large cm side roll f

## Save model

In [12]:
torch.save(model.state_dict(),
           rootDir + 'weights/instGenerator_model.pt')


NameError: name 'rootDir' is not defined

# Tensorboard visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/runs/instTrainer